# Data Cleaning

## Zweiter Teil: Unnötige Daten und Inkonsistenzen

Wann können Daten unnötig sein?
* Uninformativ/Wiederholungen
* Irrelevant
* Duplikate

## Uninformative Daten

Features können uninformativ sein, wenn viele Einträge identisch sind. Auch hier gibt es wieder keine allgemeine Regel, sondern es hängt von den Umständen ab.

Wir erkunden den Datensatz mit der willkürlichen Grenze 95% identischer Daten pro Feature:

In [15]:
import pandas as pd
df = pd.read_csv("data/sberbank-russian-housing-market/train.csv")

num_rows = len(df.index)
low_information_cols = []

for col in df.columns:
    cnts = df[col].value_counts(dropna=False)
    top_pct = (cnts/num_rows).iloc[0]

    if top_pct > 0.95:
        low_information_cols.append(col)
        print('{0}: {1:.5f}%'.format(col, top_pct*100))
        print(cnts)
        print()

oil_chemistry_raion: 99.02858%
no     30175
yes      296
Name: oil_chemistry_raion, dtype: int64

railroad_terminal_raion: 96.27187%
no     29335
yes     1136
Name: railroad_terminal_raion, dtype: int64

nuclear_reactor_raion: 97.16780%
no     29608
yes      863
Name: nuclear_reactor_raion, dtype: int64

big_road1_1line: 97.43691%
no     29690
yes      781
Name: big_road1_1line, dtype: int64

railroad_1line: 97.06934%
no     29578
yes      893
Name: railroad_1line, dtype: int64

cafe_count_500_price_high: 97.25641%
0    29635
1      787
2       38
3       11
Name: cafe_count_500_price_high, dtype: int64

mosque_count_500: 99.51101%
0    30322
1      149
Name: mosque_count_500, dtype: int64

cafe_count_1000_price_high: 95.52689%
0    29108
1     1104
2      145
3       51
4       39
5       15
6        8
7        1
Name: cafe_count_1000_price_high, dtype: int64

mosque_count_1000: 98.08342%
0    29887
1      584
Name: mosque_count_1000, dtype: int64

mosque_count_1500: 96.21936%
0    29

Was ist zu tun?

Uninformative Features können ausgeschlossen (gelöscht) werden. Dies sollte aber nicht leichtfertig geschehen.
Im Beispiel würde ich per se keins der Features ausschliessen.

## Irrelevante Daten

Dies ist eher eine qualitative Aufgabe, die Daten selber helfen uns dabei nur bedingt. Irrelevante Daten _können_ später im Rahmen der Datenanalyse ermittelt werden, also Daten, die beispielsweise für ein Machine-Learning-Modell keine Rolle spielen. Das wissen wir aber ja zum Zeitpunkt des Cleanings noch nicht!

Trotzdem können irrelevante Daten auch in der Praxis auftreten, beispielsweise bei der Kombination von Datensätzen unterschiedlicher Herkunft. Daten, die in einem Anwendungsfeld relevant sind, müssen dies nicht auch in anderen Feldern sein.


## Duplikate

Duplikate können in Zeilen und Spalten auftreten. Bei der Suche nach Duplikaten eindeutige IDs usw. ausschliessen.

Wir unterscheiden weiter zwischen vollständigen Duplikaten (Kopien) und solchen, die sich nur wenig unterscheiden - also analog zu den uninformativen Daten weiter oben.

In [16]:
# Spalte "id" muss ausgeschlossen werden, dann kann "drop_duplicates" genutzt werden
df_dedupped = df.drop('id', axis=1).drop_duplicates()

# falls was fehlt, waren es Duplikate...
print(df.shape)
print(df_dedupped.shape)

(30471, 292)
(30461, 291)


Eine Spalte fehlt - das waren wir selbst (id).
Zehn Zeilen fehlen, das waren Duplikate.

Damit sind die Duplikate auch gleich gelöscht...

Auch sich leicht unterscheidende Dateneinträge (Zeilen) können Duplikate sein.
Wie kann so etwas passieren? Beispielsweise doppelte manuelle Einträge. Wer schon mal eine Wohnung oder ein Haus gesucht hat, weiss, dass es oft identische Objekte mit leicht abgewandelten Beschreibungen auf unterschiedlichen Portalen gibt. So können über automatische oder manuelle Datenerfassung auch sich leicht unterscheidende, aber eigentlich identische Objekte in die Datensammlung gelangen.

Wir machen eine Probe: Gibt es Wohnungen mit:
* identischem Zeitstempel
* gleicher Grundfläche
* gleichem Stockwerk und Raumzahl
* gleichem Preis
* gleichem Baujahr

Dann gehen wir davon aus, dass dies Duplikate sind.

In [17]:
keys = ['timestamp', 'full_sq', 'life_sq', 'floor', 'build_year', 'num_room', 'price_doc']

df.fillna(-999).groupby(keys)['id'].count().sort_values(ascending=False).head(20)

timestamp   full_sq  life_sq  floor  build_year  num_room  price_doc
2014-04-15  134       134.0   1.0     0.0         3.0      5798496      2
2015-03-30  41        41.0    11.0    2016.0      1.0      4114580      2
2015-03-14  62       -999.0   2.0    -999.0       2.0      6520500      2
2012-09-05  43       -999.0   21.0   -999.0      -999.0    6229540      2
2014-12-09  40       -999.0   17.0   -999.0       1.0      4607265      2
2013-06-24  40       -999.0   12.0   -999.0      -999.0    4112800      2
2013-08-30  40       -999.0   12.0   -999.0       1.0      4462000      2
2013-09-23  85       -999.0   14.0   -999.0       3.0      7725974      2
2012-08-27  59       -999.0   6.0    -999.0      -999.0    4506800      2
2014-01-22  46        28.0    1.0     1968.0      2.0      3000000      2
2012-10-22  61       -999.0   18.0   -999.0      -999.0    8248500      2
2013-04-03  42       -999.0   2.0    -999.0      -999.0    3444000      2
2014-12-17  62       -999.0   9.0    -999.0

Wie wir sehen, sind die ersten 16 Einträge (nach unserer Logik) Duplikate.

Was kann man tun? Am besten die verdächtigen Daten genauer unter die Lupe nehmen und dann entscheiden, ob gelöscht werden soll:

In [18]:
# drop duplicates based on a subset of variables.

df_dedupped2 = df.drop_duplicates(subset=keys)

print(df.shape)
print(df_dedupped2.shape)

(30471, 292)
(30455, 292)


16 Einträge weniger...

## Inkonsistente Daten

Daten-Inkonsistenzen sind heimtückischer als die fehlenden oder mehrfach vorhandenen Daten, die wir bislang behandelt haben. Der Datensatz sieht OK aus, Modelle und Analysen funktionieren prinzipiell, aber können nicht ihr volles Potenzial entfalten.

Warum? Eigentlich identische Feature-Werte werden beispielsweise wegen abweichender Schreibweise als unterschiedlich gezählt. Andere Werte sind irrtümlich identisch, sollten aber eigentlich unterschiedlich sein, beispielsweise wegen abweichender Formate oder Rundung.

Damit gibt es auch kein Standard-Rezept, um alle Inkonsistenzen aufzuspüren. Hier werden einige Beispiele vorgeführt.

### Inkonsistente Schreibweise
#### Häufiges Beispiel: Gross-/Kleinschreibung



In [19]:
df['sub_area'].value_counts(dropna=False)

Poselenie Sosenskoe               1776
Nekrasovka                        1611
Poselenie Vnukovskoe              1372
Poselenie Moskovskij               925
Poselenie Voskresenskoe            713
                                  ... 
Molzhaninovskoe                      3
Poselenie Shhapovskoe                2
Poselenie Kievskij                   2
Poselenie Klenovskoe                 1
Poselenie Mihajlovo-Jarcevskoe       1
Name: sub_area, Length: 146, dtype: int64

Hier scheint kein Problem vorzuliegen.
Man kann sich aber leicht vorstellen, dass unterschiedliche Schreibweisen der Regionen vorkommen könnten.

Wie können die aufgespürt werden? Mittels Normalisierung der Schreibweise:

In [20]:
df['sub_area_lower'] = df['sub_area'].str.lower()  # neue Spalte anlegen
df['sub_area_lower'].value_counts(dropna=False)

poselenie sosenskoe               1776
nekrasovka                        1611
poselenie vnukovskoe              1372
poselenie moskovskij               925
poselenie voskresenskoe            713
                                  ... 
molzhaninovskoe                      3
poselenie shhapovskoe                2
poselenie kievskij                   2
poselenie klenovskoe                 1
poselenie mihajlovo-jarcevskoe       1
Name: sub_area_lower, Length: 146, dtype: int64

(keine Veränderung zu vorher)

### Datenformate

Häufig auftretendes Beispiel: Datumsformate
Sind oft Strings, keine Datumswerte. Manuelle Umwandlung ist notwendig.

In [21]:
df["timestamp"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 30471 entries, 0 to 30470
Series name: timestamp
Non-Null Count  Dtype 
--------------  ----- 
30471 non-null  object
dtypes: object(1)
memory usage: 238.2+ KB


Hier: object, also eigentlich ein String.

Eine Umwandlung hilft uns doppelt. Zum einen wird geprüft, ob alle Einträge gültige Datumswerte sind. Zum anderen können wir hinterher einfacher auf Zeitbereiche zugreifen (Jahre, Quartale usw.)

In [22]:
df['timestamp_dt'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d')  # explizite Formatangabe: 4 Ziffern Jahr, je 2 für Monat und Tag
df['year'] = df['timestamp_dt'].dt.year  # Explizite Speicherung eigentlich nicht notwendig, hier nur zur Verdeutlichung
df['month'] = df['timestamp_dt'].dt.month
df['weekday'] = df['timestamp_dt'].dt.weekday

print(df['year'].value_counts(dropna=False))
print()
print(df['month'].value_counts(dropna=False))

2014    13662
2013     7978
2012     4839
2015     3239
2011      753
Name: year, dtype: int64

12    3400
4     3191
3     2972
11    2970
10    2736
6     2570
5     2496
9     2346
2     2275
7     1875
8     1831
1     1809
Name: month, dtype: int64


In [23]:
df.timestamp_dt.info()

<class 'pandas.core.series.Series'>
RangeIndex: 30471 entries, 0 to 30470
Series name: timestamp_dt
Non-Null Count  Dtype         
--------------  -----         
30471 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 238.2 KB


### Kategorielle Daten

Kategorielle Daten sollten eigentlich unproblematisch sein, da sie einen Wert aus einer vorgegebenen Anzahl Kategorien einnehmen. Beispiel: PKW, LKW usw. Trotzdem können durch Fehler bei der Datenerfassung oder bei der Datenzusammenführung Fehler auftreten, die bereinigt werden müssen.

Was kann getan werden? Die Spalten mit kategoriellen Daten können untersucht werden, ob alle Werte aus den vorgegebenen Kategorien stammen. (Ohne Beispiel, im Datensatz gibt es dieses Problem nicht.)

### Weitere textuelle Daten wie Adressen usw.

Bei Texten allgemein und auch bei speziellen Texten wie beispielsweise Adresswerten treten oft Probleme auf, die auf mangelnder Standardisierung beruhen. Dies lässt sich mit Methoden des NLP beheben. Das kommt dann nächstes Jahr...

Inhalte teilweise übernommen aus: https://towardsdatascience.com/data-cleaning-in-python-the-ultimate-guide-2020-c63b88bf0a0d